# Trouver les CF sous-jacentes dans les règles
Pour trouver les CF sous-jacentes dans les règles, on commence par calculer les contextes de transformation réciproque pour chaque paire de cases, c'est à dire pour chaque famille de règles de transformation

## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [1]:
# -*- coding: utf8 -*-
import codecs,glob
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import networkx as nx
import pickle,yaml
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

### Préparation des matrices de traits

In [2]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [3]:
rep="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Longitudinales/"
rep="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/"
fichiers=glob.glob(rep+"*X-Regles.pkl")
samples=[f.rsplit("/",1)[-1].split("Regles")[0] for f in fichiers]
samples={int(s.split("-")[1]):s for s in samples}
samples

{0: 'Longitudinal-00-T10000-F3663-X-',
 1: 'Longitudinal-01-T20000-F5816-X-',
 2: 'Longitudinal-02-T30000-F7497-X-',
 3: 'Longitudinal-03-T40000-F8900-X-',
 4: 'Longitudinal-04-T50000-F10082-X-',
 5: 'Longitudinal-05-T60000-F11252-X-',
 6: 'Longitudinal-06-T70000-F12207-X-',
 7: 'Longitudinal-07-T80000-F13155-X-',
 8: 'Longitudinal-08-T90000-F14038-X-',
 9: 'Longitudinal-09-T100000-F14858-X-',
 10: 'Longitudinal-10-T110000-F15616-X-',
 11: 'Longitudinal-11-T120000-F16280-X-',
 12: 'Longitudinal-12-T130000-F16948-X-',
 13: 'Longitudinal-13-T140000-F17587-X-',
 14: 'Longitudinal-14-T150000-F18240-X-',
 15: 'Longitudinal-15-T160000-F18824-X-',
 16: 'Longitudinal-16-T170000-F19369-X-',
 17: 'Longitudinal-17-T180000-F19921-X-',
 18: 'Longitudinal-18-T190000-F20415-X-',
 19: 'Longitudinal-19-T200000-F20877-X-',
 20: 'Longitudinal-20-T210000-F21393-X-',
 21: 'Longitudinal-21-T220000-F21883-X-',
 22: 'Longitudinal-22-T230000-F22343-X-',
 23: 'Longitudinal-23-T240000-F22786-X-',
 24: 'Longitudi

In [4]:
sample=samples[50]
fRulesPMS="Regles.pkl"
fRulesPMO="Morphomes-Regles.pkl"

# Classe pour la gestion des patrons, des classes et des transformations

In [5]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print "le numéro de forme n'est pas dans [1,2]",n

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=False):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
#                if debug: 
#                    print (forme, file=logfile)
#                    print ("pas de classe",idClasseForme, file=logfile)
#                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
#        else:
#            if debug:
#                print (forme, file=logfile) 
#                print ("pas de patron", file=logfile)
        return sortieForme
        

## Ouvrir les fichiers de règles

In [6]:
def openRules(numero,typeRules="PMS"):
    rules=None
    if typeRules=="PMS" and numero in samples:
        with open(rep+samples[numero]+fRulesPMS, 'rb') as input:
            rules = pickle.load(input)
    elif typeRules=="PMO" and numero in samples:
        with open(rep+samples[numero]+fRulesPMO, 'rb') as input:
            rules = pickle.load(input)
    return rules

## Conversion Positions <=> Regex

In [7]:
def getRegexPositions(positions):
    return "".join([p if len(p)<2 else "[%s]"%''.join(sorted(p)) for p in positions])

In [8]:
def getPositionsRegex(regex):
    if regex=="": return []
    result=[]
    regex=re.sub(ur"[()]","",regex.replace(".*","X"))
    chunks=[c for c in re.split(ur"(\[[^\]]+\])",regex) if c!=""]
    if chunks[0].startswith("^"): chunks[0]=chunks[0][1:]
    if chunks[-1].endswith("$"): chunks[-1]=chunks[-1][:-1]
    for chunk in chunks:
        if chunk.startswith("["): 
            result.append(chunk.replace("[","").replace("]",""))
        else:
            result.extend(chunk)
    return result

## Calcul des intersections

In [9]:
def getIntersectionPos(l1,l2):
    l=set()
    if l1=="X" or l1==".":
        l=set(l2)
    elif l2=="X" or l2==".":
        l=set(l1)
    else:
        l=set(l1)&set(l2)
    return l

def getIntersectionRegex(gP1,gP2,debug=False):
    p1=gP1[:]
    p2=gP2[:]
    pMin,pMax=sorted([p1,p2],key=len)
    temp=[]
    pMin.reverse()
    pMax.reverse()
#    print p1,p2
    for i in range(len(pMin)):
        l=getIntersectionPos(pMin[i],pMax[i])
        if l:
#            print ".",
            temp.append(l)
        else:
            return []
    if len(pMax)>len(pMin):
        if pMin[-1]=="X":
            for i in range(len(pMin),len(pMax)):
                temp.append(set(pMax[i]))
        elif pMax[len(pMin)]!="X":
            return []
    temp.reverse()
    result=[]
    for c in temp:
        result.append("".join(sorted(c)))
#    if "" in result:
#        result=""
    return result

## Calcul des transformations

In [10]:
def transformeExp(gPositions,patron):
    result=[]
    positions=gPositions[:]
    positions.reverse()
    e0,s0=patron.split("-")
    e1=re.split(ur"(\.)",e0)
    s1=re.split(ur"(\.)",s0)
    e1.reverse()
    s1.reverse()
#    print e1, s1, positions
    lPosition=0
    for nChunk,chunk in enumerate(e1):
#        print "chunk",chunk, nChunk, "pos", lPosition
        if chunk!=".":
            if chunk:
                for nLettre,lettre in enumerate(chunk):
    #                print "lettre",nLettre,lettre,"pos",lPosition
                    if nLettre==0: result.extend(s1[nChunk][::-1])
                    lPosition+=1
            else:
                result.extend(s1[nChunk][::-1])
        else:
#            print "lettre",positions[lPosition],"pos",lPosition
            result.append(positions[lPosition])
            lPosition+=1
#    print result
    if len(positions)>lPosition:
        for i in range(lPosition,len(positions)):
            result.append(positions[i])
    elif positions[-1]==u"X":
        result.append(u"X")
    result.reverse()
    return [r for r in result if r!=""]
        

## Calcul des contraintes

In [11]:
def nouvellesContraintes(contraintes,paire,addEdge=False,gPatrons=[]):
    if not gPatrons:
        patrons=patronsConsideres[paire]
    else:
        patrons=gPatrons[paire]
    result1=[]
#    result2={}
    for r in contraintes:
        for p in patrons:
            regexRegle=getPositionsRegex(patrons[p])
            regex=getIntersectionRegex(r,regexRegle)
            if regex:
                trans=transformeExp(regex,p)
                if trans and trans not in result1:
                    result1.append(trans)
#                    result2[p]=(regex,trans)
                    if addEdge:
                        pointA=paire[0]+":"+getRegexPositions(regex)
                        pointB=paire[1]+":"+getRegexPositions(trans)
                        if debug: print pointA+"-"+pointB
                        reseauTrans.add_edge(pointA,pointB,key=p)
                    if debug:
                        print "TRANS",r, p, regex
                        print "=>",trans
    if result1:
        return result1
    else:
        return contraintes

def getContraintes(A,B,gPatrons=[]):
    aller=nouvellesContraintes([[u"X"]],(A,B),gPatrons=gPatrons)
    retour=nouvellesContraintes(aller,(B,A),gPatrons=gPatrons)
    return retour

def getListesContraintes(dictA,A,nodes,gPatrons=[]):
    lDictA=dict(dictA)
    for node in nodes:
        lDictA[node]=getContraintes(A,node,gPatrons=gPatrons)
    return lDictA

def mergeContraintesAB(contraintesA,contraintesB):
    result=[]
    for a in contraintesA:
        for b in contraintesB:
            intersection=getIntersectionRegex(a,b)
            if intersection and intersection not in result:
                result.append(intersection)
    return result

In [12]:
def mergeContraintes(dictA):
    cases=dictA.keys()
    contraintes=dictA[cases[0]]
    print len(cases)
    for case in cases[1:]:
#        print ".",
        if dictA[case]:
            contraintes=mergeContraintesAB(contraintes,dictA[case])
    return contraintes

In [13]:
def mergeContraintesConflate(dictA):
    cases=dictA.keys()
    contraintes=dictA[cases[0]]
    autresContraintes=[]
    for case in cases[1:]:
        for c in dictA[case]:
            if c not in autresContraintes:
                autresContraintes.append(c)
    contraintes=mergeContraintesAB(contraintes,autresContraintes)
    return contraintes

In [14]:
def simplificationContraintesConsolideesCase(listeContraintes):
    modif=False
    lContraintes=listeContraintes[:]
    for c1,c2 in it.combinations(listeContraintes,2):
#        print c1,c2
        intersection=getIntersectionRegex(c1,c2)
        if intersection:
            if intersection==c1 and c1 in lContraintes:
                lContraintes.remove(c1)
            elif intersection==c2 and c2 in lContraintes:
                lContraintes.remove(c2)
    return lContraintes

def simplificationContraintesConsolidees(dictListeContraintes):
    lDictListeContraintes=dictListeContraintes.copy()
    for case in lDictListeContraintes:
        lDictListeContraintes[case]=simplificationContraintesConsolideesCase(lDictListeContraintes[case])
    return lDictListeContraintes

## Choix des Patrons à considérer
Deux façons de traiter les *patrons* associés à l'échantillon qui a fourni les règles (*rules*)
- avec contextes
- sans contextes

In [15]:
def getPatronsConsideres(rules,avecContexte=False):
    patronsConsideres={}
    patronsAvecContexte={}
    patronsSansContexte={}
    for paire in rules.keys():
        patronsAvecContexte[paire]=rules[paire].patrons
        for r in patronsAvecContexte[paire]:
            if paire not in patronsSansContexte:
                patronsSansContexte[paire]={}
            transformation=r.split("-")[0]
            patronsSansContexte[paire][r]=u"^(.*)%s$"%transformation
    if not avecContexte:
        patronsConsideres=patronsSansContexte
    else:
        patronsConsideres=patronsAvecContexte
    return patronsConsideres

In [31]:
numero=50
typeRules="PMO"
avecContexte=False

In [16]:
rules=openRules(numero,typeRules=typeRules)
patronsConsideres=getPatronsConsideres(rules,avecContexte=avecContexte)

In [17]:
patronsConsideres

{(u'pI2P', u'ps1P'): {u'Et-asj\xf4': u'^(.*)Et$',
  u'e-j\xf4': u'^(.*)e$',
  u'e-\xf4': u'^(.*)e$'},
 (u'ai1S', u'pc2P'): {u'-rje': u'^(.*)$',
  u'6.E-E.6rje': u'^(.*)6.E$',
  u'E-6rje': u'^(.*)E$',
  u'EJi-\xeadrije': u'^(.*)EJi$',
  u'HE-rje': u'^(.*)HE$',
  u'alE-irje': u'^(.*)alE$',
  u'fy-s6rje': u'^(.*)fy$',
  u'i-6rje': u'^(.*)i$',
  u'i-Erje': u'^(.*)i$',
  u'i-rije': u'^(.*)i$',
  u'i-\xe2drije': u'^(.*)i$',
  u'ly-drije': u'^(.*)ly$',
  u'y-6vrije': u'^(.*)y$',
  u'y-Erje': u'^(.*)y$',
  u'y-Orje': u'^(.*)y$',
  u'y-warje': u'^(.*)y$',
  u'\xea-j\xeadrije': u'^(.*)\xea$'},
 (u'is1P', u'ii2P'): {u'fysj\xf4-Etje': u'^(.*)fysj\xf4$',
  u'is.\xf4-6z.e': u'^(.*)is.\xf4$',
  u'ys.\xf4-av.e': u'^(.*)ys.\xf4$'},
 (u'ai2S', u'fi2P'): {u'-re': u'^(.*)$',
  u'6.a-E.6re': u'^(.*)6.a$',
  u'EJi-\xeadre': u'^(.*)EJi$',
  u'Eky-ivre': u'^(.*)Eky$',
  u'Ha-re': u'^(.*)Ha$',
  u'a-6re': u'^(.*)a$',
  u'aJi-\xeadre': u'^(.*)aJi$',
  u'ala-ire': u'^(.*)ala$',
  u'fy-s6re': u'^(.*)fy$',
  u'i-6

## Définition des cases à prendre en compte

In [18]:
paradigmeCases=list(set([p[0] for p in rules.keys()]))
#paradigmeCases=[c for c in paradigmeCases if not "1" in c and not "2" in c and not "ai" in c and not "is" in c]
selectionCases=paradigmeCases
#selectionCases=testCells
#selectionCases=list(set([e[0] for e in patronsSansContexte]))
print selectionCases

[u'ii1P', u'pP', u'is1S', u'ii1S', u'ppMP', u'ai1P', u'ai1S', u'pc1S', u'pi2S', u'inf', u'pi2P', u'ppFP', u'ps1P', u'is2P', u'ps1S', u'pI2S', u'fi2P', u'fi2S', u'pI2P', u'ps2P', u'ai2P', u'ai2S', u'pc2P', u'ii2P', u'pi1S', u'pi1P', u'pi3P', u'pc1P', u'is1P', u'ai3P', u'fi1P', u'fi1S', u'pI1P']


# Contraintes en étoile
Pour calculer les CF abstraites, on calcule pour chaque case, les contraintes imposées par les autres et on les combine.
- chaque case est susceptible d'imposer des éléments à ses correspondantes
 - on commence par calculer les contraintes en partant d'une représentation complètement sous-spécifiée : X
- pour chaque correspondante, on calcule tous les éléments imposés par les autres et on fait une consolidation des contraintes : contraintesConsolidées
 - avec les contraintes consolidées, on recalcule les contraintes imposées sur les correspondants
 - et on reconsolide le tout
- avec ces contraintes qui sont représentatives de l'ensemble, on peut faire le réseau qui relie les différentes représentations par les transformations
 - les relations réciproques sont captées dans un graphe non-orienté
 - les cliques du graphe non-orienté sont des classes flexionnelles
  - les classes flexionnelles complètes sont celles qui couvrent le paradigme


In [19]:
eliminationCases=["is%d%s"%(i,n) for i in range(1,4) for n in "SP"]+["ppFS","ppFP"]
eliminationCases+=["ai%d%s"%(i,n) for i in range(1,4) for n in "SP"]
selectionCases=[c for c in selectionCases if not c in eliminationCases]
contraintesCase={c:{} for c in selectionCases}
contraintesConsolidees={}

### Calculer les contraintes 
- pour toutes les cases a 
 - en partant des différentes cases b

In [20]:
def getContraintesCases(consolidation=[]):
    lContraintes={c:{} for c in selectionCases}
    if not consolidation:
        consolidation={c:[[u"X"]] for c in lContraintes}
    for a in lContraintes:
        for b in [c for c in lContraintes if c!=a]:
            if (b,a) in patronsConsideres:
                lContraintes[a][b]=nouvellesContraintes(consolidation[b],(b,a))
    return lContraintes

### Unifier les contraintes
- de toutes les cases

In [21]:
def unifyContraintesCases(contraintes,alt=False):
    lContraintesConsolidees={}
    for c in contraintes:
        print c,
        if alt:
            lContraintesConsolidees[c]=mergeContraintesAlt(contraintes[c])
        else:
            lContraintesConsolidees[c]=mergeContraintes(contraintes[c])
    print
    return lContraintesConsolidees

## Simplifier les contraintesConsolidees

In [22]:
%%time 
contraintesCase=getContraintesCases()

CPU times: user 285 ms, sys: 15.3 ms, total: 300 ms
Wall time: 299 ms


In [23]:
%time contraintesConsolidees=unifyContraintesCases(contraintesCase)
#%time contraintesConsolideesAlt=unifyContraintesCases(contraintesCase,alt=True)

ii1P 23
pP 23
ii1S 23
ppMP 23
inf 23
ps1P 23
ps1S 23
pi2S 23
pi2P 23
pI2S 23
fi2P 23
fi2S 23
pI2P 23
ps2P 23
pc2P 23
ii2P 23
pi1S 23
pi1P 23
pi3P 23
pc1P 23
pc1S 23
fi1P 23
fi1S 23
pI1P 23

CPU times: user 4.51 s, sys: 558 ms, total: 5.07 s
Wall time: 4.67 s


In [24]:
%%time 
contraintesCase=getContraintesCases(contraintesConsolidees)

CPU times: user 25.5 s, sys: 687 ms, total: 26.2 s
Wall time: 25.7 s


In [25]:
%%time 
contraintesConsolidees=unifyContraintesCases(contraintesCase)

ii1P 23
pP 23
ii1S 23
ppMP 23
inf 23
ps1P 23
ps1S 23
pi2S 23
pi2P 23
pI2S 23
fi2P 23
fi2S 23
pI2P 23
ps2P 23
pc2P 23
ii2P 23
pi1S 23
pi1P 23
pi3P 23
pc1P 23
pc1S 23
fi1P 23
fi1S 23
pI1P 23

CPU times: user 2h 33min 41s, sys: 4min 36s, total: 2h 38min 17s
Wall time: 2h 34min 49s


In [27]:
reseauTrans=nx.DiGraph()
for k in contraintesCase:    
    for elt in [c for c in contraintesCase if c!=k]:
#        print k,elt,contraintesConsolidees[k]
        if (k,elt) in patronsConsideres:
            nouvellesContraintes(contraintesConsolidees[k],(k,elt),addEdge=True)

In [28]:
reseauSimTrans=nx.Graph()
for edge in reseauTrans.edges():
    a,b=sorted(edge)
    if reseauTrans.has_edge(a,b) and reseauTrans.has_edge(b,a) and not reseauSimTrans.has_edge(a,b):
        reseauSimTrans.add_edge(a,b)
cliques=list(nx.find_cliques(reseauSimTrans))

In [29]:
def simplifyCliques(cliques):
    nouvellesCliques=[]
    for c in cliques:
        cases=[e.split(":")[0] for e in c]
        formes=[e.split(":")[1] for e in c]
        maxL=len(min(formes))
        if maxL>1 and formes[0][0]==u"X":
            cible="X"
            for i in range(1,maxL):
                entete=formes[0][:i+1]
                if all([e[:i+1]==entete for e in formes]):
                    cible=entete
                else:
                    break
            formes=[f.replace(cible,u"X") for f in formes]
        nouvelleClique=set([cases[i]+":"+formes[i] for i in range(len(cases))])
        if nouvelleClique not in nouvellesCliques:
            nouvellesCliques.append(nouvelleClique)
    return nouvellesCliques

In [ ]:
nouvellesCliques=simplifyCliques(cliques)

In [36]:
print len(selectionCases),numero,typeRules,avecContexte
seuil=10
for c in nouvellesCliques:
    if len(c)>len(selectionCases)-1-seuil and "inf:XEr" in c:
        print len(c), ",".join(sorted(c))

24 50 PMO False
14 ii1P:X6zjô,ii1S:X6zE,ii2P:X6zje,inf:XEr,pI1P:X6zô,pI2P:XEt,pP:X6zâ,pi1P:X6zô,pi1S:XE,pi2P:XEt,pi2S:XE,ps1P:Xasjô,ps1S:Xas,ps2P:Xasje
15 ii1P:X6zjô,ii1S:X6zE,ii2P:X6zje,inf:XEr,pI1P:X6zô,pI2P:XEt,pI2S:XE,pP:X6zâ,pi1P:X6zô,pi1S:XE,pi2P:XEt,pi2S:XE,ps1P:Xasjô,ps1S:Xas,ps2P:Xasje
